## 환경설정 및 데이터 세팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # ▶ 한글 폰트 설정하기
!pip install koreanize_matplotlib -q

# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.3 MB/s eta 0:00:00


In [ ]:
# 필요한 라이브러리 불러오기
import re # 정규표현모듈
import copy
import pandas as pd
import numpy as np
import ast
from collections import Counter
from datetime import datetime
from urllib.parse import urlparse, parse_qs

In [ ]:
# 시각화 임포트
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.gridspec import GridSpec
'''
# ✅ Seaborn 테마 설정
sns.set_theme(
    style='white',  # 배경 스타일 ('darkgrid', 'whitegrid', 'dark', 'white', 'ticks' 가능)
    font='AppleGothic',  # 폰트 설정 (Mac: 'Apple Gothic'), (win: 'BM Hanna Pro')
    rc={
        'axes.unicode_minus': False,  # 마이너스 기호 깨짐 방지
        'figure.figsize': (12, 6),    # 기본 그래프 크기 설정
        'axes.labelsize': 14,         # 축 레이블 크기
        'xtick.labelsize': 12,        # x축 눈금 크기
        'ytick.labelsize': 12,        # y축 눈금 크기
        'legend.fontsize': 12,        # 범례 폰트 크기
        'axes.titleweight': 'bold',   # 제목 굵게 표시
        'axes.titlesize': 16          # 제목 크기 설정
    }
)
'''
# ✅ Jupyter Notebook 그래프 선명하게 출력
%config InlineBackend.figure_format = 'retina'

# ✅ Pandas 출력 설정 (데이터프레임 가독성 향상)
pd.set_option("display.max_columns", None)  # 모든 컬럼 출력
pd.set_option('display.width', None)
# pd.set_option('display.float_format', '{:.5f}'.format)  # 소수점 5자리까지 표시

# ✅ 경고 메시지 무시 (불필요한 Warning 제거)
warnings.filterwarnings("ignore")

# ✅ Jupyter에서 외부 모듈 변경 사항 자동 반영
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
plt.rcParams['axes.unicode_minus'] = False
import matplotlib.cm as cm
from collections import Counter

- df는 각자 폴더의 path 경로 붙여넣기

In [ ]:
company = pd.read_csv('/content/drive/MyDrive/7팀_중급프로젝트/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/Company.csv')
job = pd.read_csv('/content/drive/MyDrive/7팀_중급프로젝트/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/Job.csv')
application = pd.read_csv('/content/drive/MyDrive/7팀_중급프로젝트/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/Application.csv')

## 전처리

### 분석 코드

In [ ]:
# 5/24 이후 로그에서 데이터 전처리
# 공유드라이브 > 7팀 > 전처리데이터 > cleaned_log_23_df.csv 다운
cleaned_log_23_df = pd.read_csv("/content/drive/MyDrive/7팀_중급프로젝트/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/cleaned_log_23_df.csv", low_memory=False)
cleaned_log_23_df

,timestamp,user_uuid,URL,level_0,level_1,level_2,level_3,level_4,level_5,level_6,utm,utm_source,utm_campaign,utm_medium,utm_content,keyword,general_query,response_code,method,create_year,create_month,create_day,create_day_of_week
0,2023-05-24 00:00:12,aea164b6-21e1-4b82-8f58-849b1f33892e,jobs/id/apply/step2,jobs,id,apply,step2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-05,2023-05-24,Wednesday
1,2023-05-24 00:00:12,aea164b6-21e1-4b82-8f58-849b1f33892e,api/jobs/id/apply/step1,api,jobs,id,apply,step1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,POST,2023,2023-05,2023-05-24,Wednesday
2,2023-05-24 00:00:13,aea164b6-21e1-4b82-8f58-849b1f33892e,api/users/id/experience/form?type=apply,api,users,id,experience,form,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type=apply,200,POST,2023,2023-05,2023-05-24,Wednesday
3,2023-05-24 00:00:13,aea164b6-21e1-4b82-8f58-849b1f33892e,api/users/id/experience/form?type=apply,api,users,id,experience,form,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type=apply,200,POST,2023,2023-05,2023-05-24,Wednesday
4,2023-05-24 00:00:14,a10979c3-cff0-4017-806c-8d4d3b6c1b1c,api/users/notifications/mark_read?id=6006650&_...,api,users,notifications,mark_read,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,id=6006650&_=1684886392371,200,GET,2023,2023-05,2023-05-24,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381152,2023-12-31 14:53:38,17bea2bb-c87c-488c-9c64-44eb0383c6d6,@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-31,Sunday
3381153,2023-12-31 14:53:39,17bea2bb-c87c-488c-9c64-44eb0383c6d6,@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-31,Sunday
3381154,2023-12-31 14:53:41,17bea2bb-c87c-488c-9c64-44eb0383c6d6,api/users/id/template,api,users,id,template,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,POST,2023,2023-12,2023-12-31,Sunday
3381155,2023-12-31 14:55:11,17bea2bb-c87c-488c-9c64-44eb0383c6d6,@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-31,Sunday


In [ ]:
cleaned_log_23_df.isna().sum()

,0
timestamp,0
user_uuid,0
URL,133901
level_0,134213
level_1,655675
level_2,1110709
level_3,2014360
level_4,3039268
level_5,3349641
level_6,3380891


In [ ]:
# GET 메소드만 추출
get_grouped = cleaned_log_23_df[cleaned_log_23_df['method'] == 'GET']

# response 코드 처리
filter_df = get_grouped[get_grouped['response_code'].isin([200, 301, 302])]

# 'path' 컬럼에 'api'가 포함되어 있는 행들을 제외합니다.
filter_df = filter_df[~filter_df['URL'].str.contains('api', na=False)]
filter_df.reset_index(drop=True, inplace=True)

filter_df

,timestamp,user_uuid,URL,level_0,level_1,level_2,level_3,level_4,level_5,level_6,utm,utm_source,utm_campaign,utm_medium,utm_content,keyword,general_query,response_code,method,create_year,create_month,create_day,create_day_of_week
0,2023-05-24 00:00:12,aea164b6-21e1-4b82-8f58-849b1f33892e,jobs/id/apply/step2,jobs,id,apply,step2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-05,2023-05-24,Wednesday
1,2023-05-24 00:00:15,a10979c3-cff0-4017-806c-8d4d3b6c1b1c,@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-05,2023-05-24,Wednesday
2,2023-05-24 00:00:26,735b3857-387b-4642-8e6e-ce539e9e5724,@user_id/applications,@user_id,applications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-05,2023-05-24,Wednesday
3,2023-05-24 00:00:26,735b3857-387b-4642-8e6e-ce539e9e5724,@user_id/applications,@user_id,applications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-05,2023-05-24,Wednesday
4,2023-05-24 00:00:28,aea164b6-21e1-4b82-8f58-849b1f33892e,jobs/id/apply/step3,jobs,id,apply,step3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-05,2023-05-24,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387002,2023-12-31 14:48:43,c5ff0e9a-be09-4d21-9a76-53d294256444,@user_id/applications,@user_id,applications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-31,Sunday
1387003,2023-12-31 14:48:43,c5ff0e9a-be09-4d21-9a76-53d294256444,continue?next=/@yellowatkk/applications,continue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,next=/@yellowatkk/applications,302,GET,2023,2023-12,2023-12-31,Sunday
1387004,2023-12-31 14:53:38,17bea2bb-c87c-488c-9c64-44eb0383c6d6,@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-31,Sunday
1387005,2023-12-31 14:53:39,17bea2bb-c87c-488c-9c64-44eb0383c6d6,@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-31,Sunday


In [ ]:
# filter_df.to_csv('*/final_project_log.csv')

In [ ]:
# 총 유저 DF
# 공유드라이브 > 7팀 > 전처리데이터폴더 > final_project_log.csv 다운
total_user = pd.read_csv('/content/drive/MyDrive/7팀_중급프로젝트/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/final_project_log.csv')

In [ ]:
total_user['timestamp'] = pd.to_datetime(total_user['timestamp'])
total_user.tail()

,timestamp,user_uuid,URL,level_0,level_1,level_2,level_3,level_4,level_5,level_6,utm,utm_source,utm_campaign,utm_medium,utm_content,keyword,general_query,response_code,method,create_year,create_month,create_day,create_day_of_week,user_type
1381042,2023-12-29 04:19:45,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/jobs/id/apply/step2,jobs,id,apply,step2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-29,Friday,newbie
1381043,2023-12-29 04:19:54,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/jobs/id/apply/step3,jobs,id,apply,step3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-29,Friday,newbie
1381044,2023-12-29 04:22:34,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/jobs?job=1,jobs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,job=1,200,GET,2023,2023-12,2023-12-29,Friday,newbie
1381045,2023-12-29 04:22:40,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/@user_id/applications,@user_id,applications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-29,Friday,newbie
1381046,2023-12-29 04:22:51,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/@user_id/applications,@user_id,applications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-29,Friday,newbie


- 세션, path 분리

In [ ]:
# 사용자의 이름 및 시간 순서로 정렬
filtered_df = total_user.sort_values(['user_uuid', 'timestamp'])

# 이전 행의 시간과의 차이를 분단위로 계산
filtered_df['time_diff'] = filtered_df.groupby('user_uuid')['timestamp'].diff().dt.total_seconds() / 60

# 시간의 차이가 30분을 초과하거나 이전의 값이 Nan값일 경우 새로운 세션으로 간주
filtered_df['new_session'] = ((filtered_df['time_diff'] > 30) | filtered_df['time_diff'].isna())

# 사용자별로 새로운 세션이 True일경우 그 시간
filtered_df['session_id'] = filtered_df.groupby('user_uuid')['new_session'].cumsum()

# URL의 path 구분
filtered_df['path'] = filtered_df['URL'].str.split('?').str[0].sort_index()

filtered_df.tail()

,timestamp,user_uuid,URL,level_0,level_1,level_2,level_3,level_4,level_5,level_6,utm,utm_source,utm_campaign,utm_medium,utm_content,keyword,general_query,response_code,method,create_year,create_month,create_day,create_day_of_week,user_type,time_diff,new_session,session_id,path
1100423,2023-12-18 15:04:22,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,4380.650000,True,13,main/@user_id
1100431,2023-12-18 15:05:58,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,1.600000,False,13,main/@user_id
1100432,2023-12-18 15:06:35,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,0.616667,False,13,main/@user_id
1100433,2023-12-18 15:06:36,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,0.016667,False,13,main/@user_id
1100435,2023-12-18 15:06:46,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/jobs,jobs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,0.166667,False,13,main/jobs


In [ ]:
new_user = filtered_df.query('user_type == "newbie"')
new_user

,timestamp,user_uuid,URL,level_0,level_1,level_2,level_3,level_4,level_5,level_6,utm,utm_source,utm_campaign,utm_medium,utm_content,keyword,general_query,response_code,method,create_year,create_month,create_day,create_day_of_week,user_type,time_diff,new_session,session_id,path
1144271,2023-06-21 11:03:51,000b55ad-5c86-438b-a5b0-9ea3a2917789,main/signup/step3/done,signup,step3,done,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-06,2023-06-21,Wednesday,newbie,NaN,True,1,main/signup/step3/done
1144272,2023-06-21 11:03:58,000b55ad-5c86-438b-a5b0-9ea3a2917789,main/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-06,2023-06-21,Wednesday,newbie,0.116667,False,1,main/
1144273,2023-06-21 11:04:04,000b55ad-5c86-438b-a5b0-9ea3a2917789,main/suggest?q=,suggest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,q=,200,GET,2023,2023-06,2023-06-21,Wednesday,newbie,0.100000,False,1,main/suggest
1144274,2023-06-21 11:04:07,000b55ad-5c86-438b-a5b0-9ea3a2917789,main/suggest?q=ㅇ,suggest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,q=ㅇ,200,GET,2023,2023-06,2023-06-21,Wednesday,newbie,0.050000,False,1,main/suggest
1144275,2023-06-21 11:04:09,000b55ad-5c86-438b-a5b0-9ea3a2917789,main/suggest?q=ㅋ,suggest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,q=ㅋ,200,GET,2023,2023-06,2023-06-21,Wednesday,newbie,0.033333,False,1,main/suggest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381042,2023-12-29 04:19:45,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/jobs/id/apply/step2,jobs,id,apply,step2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-29,Friday,newbie,0.066667,False,8,main/jobs/id/apply/step2
1381043,2023-12-29 04:19:54,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/jobs/id/apply/step3,jobs,id,apply,step3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-29,Friday,newbie,0.150000,False,8,main/jobs/id/apply/step3
1381044,2023-12-29 04:22:34,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/jobs?job=1,jobs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,job=1,200,GET,2023,2023-12,2023-12-29,Friday,newbie,2.666667,False,8,main/jobs
1381045,2023-12-29 04:22:40,ff7b5679-a7f2-4675-99ca-8055c0fec8c0,main/@user_id/applications,@user_id,applications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-29,Friday,newbie,0.100000,False,8,main/@user_id/applications


In [ ]:
old_user = filtered_df.query('user_type == "oldbie"')
old_user

,timestamp,user_uuid,URL,level_0,level_1,level_2,level_3,level_4,level_5,level_6,utm,utm_source,utm_campaign,utm_medium,utm_content,keyword,general_query,response_code,method,create_year,create_month,create_day,create_day_of_week,user_type,time_diff,new_session,session_id,path
1120543,2023-12-24 11:54:38,000681b4-bb66-4ccf-9786-f0066d13a34e,main/password_reset?code=d0308579-64c7-49b7-be...,password_reset,NaN,NaN,NaN,NaN,NaN,NaN,utm_source=notification&utm_medium=email&utm_c...,notification,password_reset,email,reset_button,NaN,code=d0308579-64c7-49b7-be8c-56227a5be083,200,GET,2023,2023-12,2023-12-24,Sunday,oldbie,NaN,True,1,main/password_reset
1120544,2023-12-24 11:54:43,000681b4-bb66-4ccf-9786-f0066d13a34e,main/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-24,Sunday,oldbie,0.083333,False,1,main/
1120545,2023-12-24 11:54:44,000681b4-bb66-4ccf-9786-f0066d13a34e,main/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-24,Sunday,oldbie,0.016667,False,1,main/
1120551,2023-12-24 11:55:42,000681b4-bb66-4ccf-9786-f0066d13a34e,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-24,Sunday,oldbie,0.966667,False,1,main/@user_id
1120562,2023-12-24 11:57:52,000681b4-bb66-4ccf-9786-f0066d13a34e,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-24,Sunday,oldbie,2.166667,False,1,main/@user_id
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100423,2023-12-18 15:04:22,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,4380.650000,True,13,main/@user_id
1100431,2023-12-18 15:05:58,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,1.600000,False,13,main/@user_id
1100432,2023-12-18 15:06:35,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,0.616667,False,13,main/@user_id
1100433,2023-12-18 15:06:36,ffff25ca-c1d7-4fc2-891b-b0df92f95092,main/@user_id,@user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,GET,2023,2023-12,2023-12-18,Monday,oldbie,0.016667,False,13,main/@user_id


In [ ]:
# job과 app 테이블 연결
job_app_merge = pd.merge(job, application, on='job_uuid')
job_app_merge = job_app_merge.drop(columns=['Unnamed: 0_x', 'company_uuid_x', 'Unnamed: 0_y', 'cdate_y', 'company_uuid_y'])
job_app_merge = job_app_merge.rename(columns={'cdate_x': 'cdate'})
job_app_merge.tail()

,cdate,mdate,job_field,career_type_string,start_date,end_date,allow_remote,can_show_salary,job_uuid,date,user_uuid,application_uuid
331183,2022-06-07 05:59:26,2022-06-25 07:56:33,SW 개발,경력,NaN,2022-06-25,1,1,03f1d118-22e3-4659-aa66-565d3e963176,2022-06-07 05:59:26,2ece8d7d-79b6-4522-928b-8dbae8843fab,f31f5a32-2143-4c48-baeb-bee57c20551b
331184,2022-06-07 05:59:26,2022-06-25 07:56:33,SW 개발,경력,NaN,2022-06-25,1,1,03f1d118-22e3-4659-aa66-565d3e963176,2022-06-07 05:59:26,2ece8d7d-79b6-4522-928b-8dbae8843fab,31c852aa-25e2-433a-b587-f96d545f40d0
331185,2022-06-07 05:59:26,2022-06-25 07:56:33,SW 개발,경력,NaN,2022-06-25,1,1,03f1d118-22e3-4659-aa66-565d3e963176,2022-06-07 05:59:26,d1e5dbd8-50fd-4e8f-8ce8-f091a7ce3110,ac4fee9c-40db-4d7a-acbd-bc608ef76781
331186,2022-06-07 05:59:26,2022-06-25 07:56:33,SW 개발,경력,NaN,2022-06-25,1,1,03f1d118-22e3-4659-aa66-565d3e963176,2022-06-07 05:59:26,36babb9f-5ddc-48af-afc5-0e43a2311a31,21711855-29d6-4d1f-9552-27728ef09921
331187,2022-06-07 05:59:26,2022-06-25 07:56:33,SW 개발,경력,NaN,2022-06-25,1,1,03f1d118-22e3-4659-aa66-565d3e963176,2022-06-07 05:59:26,8a669e51-4219-44e1-8584-e3330304468b,3becf3eb-dac6-4c24-9cbc-e31729413984
